In [1]:
import pandas as pd
train = pd.read_pickle("../data/wakati_train.pkl")
test = pd.read_pickle("../data/wakati_test.pkl")
df = pd.concat([train, test])

In [2]:
df.head()

,ncode,title_wakati,story_wakati,keyword_wakati
0,N7588B,暗い 窓辺 皇帝 北 宮,新しく 手直し 宇宙 開拓 地 入植 農家 子供 小人 生まれる 両親 子 受け入れよう し...,ファンタジー 天使 小人 猫
1,N8726B,仮想現実 ゲームヴァルハラ,ゲームセンター 片隅 置か 律儀 リアル ゲームヴァルハラ プレイ 重ねる たび たち 魅了...,仮想現実 ゲーム バトル 戦 乙女 高校生 学校 学園 アンドロイドライトエンターテイメント...
2,N5481C,海 竜騎兵,海軍 中 存在 クジラ 部隊 竜騎兵 主人公 一員 騎乗 鯨 間 結ば 絆 強く 主人公 戦...,ヒーロー 冒険 海軍 深海 海中 戦闘
3,N6260C,宇宙 狂 時代,はるか 未来 宇宙船 メーカー ペガサス 才 社長 あり 美少女 ある ミリィ 前代未聞 宇...,冒険 ロケットロボットサイボーグレースコメディエンターテイメント 少女 友情 少年 未来 宇宙
4,N6903C,クライムズクライシス,アメリカ 大富豪 テッド マクシミリアン 心筋 梗塞 患っ あの世 旅立っ 弁護士 開い 遺...,ゲーム フェア 平和 権力 ドキドキシリアスハッピーエンド 国家 民族 友情 サラリーマン ...


In [8]:
# TI-IDFを計算する
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

text_col = "story"

model = TfidfVectorizer()
wakati_df = model.fit_transform(df[text_col + "_wakati"])
print(wakati_df.shape)

(48522, 75774)


In [6]:
# 次元圧縮
svd = TruncatedSVD(64)
svd.fit(wakati_df)
tfidf_svd = svd.transform(wakati_df)
print(tfidf_svd.shape)

story_tfidf_svd_train = story_tfidf_svd[:40000]
story_tfidf_svd_test = story_tfidf_svd[40000:]

(48522, 64)


In [7]:
story_train_df = pd.DataFrame(story_tfidf_svd_train)
story_test_df = pd.DataFrame(story_tfidf_svd_test)

for col_name in story_train_df.columns:
    story_train_df = story_train_df.rename(
        columns={col_name: f"tfidf_story_{col_name}"})

for col_name in story_test_df.columns:
    story_test_df = story_test_df.rename(
        columns={col_name: f"tfidf_story_{col_name}"})

FileNotFoundError: [Errno 2] No such file or directory: '../tdidf/train_story_tfidf.npy'